In [1]:
%pip install -q -r ../../requirements.txt


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
import os
print(load_dotenv('../../.env'))
print(os.environ['LANGSMITH_PROJECT'])
SERVER_URL="http://localhost:9000"

True
agentic-ops


## Server Deployment
- server file: agent.py
- venv is active
- `sudo kill -9 $(lsof -t -i:9000)`
- `langgraph dev --port 9000 --host 0.0.0.0`

## LangGraph SDK

In [3]:
from langgraph_sdk import get_client
client = get_client(url=SERVER_URL)

In [4]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

thread = await client.threads.create()
input_message = HumanMessage(content="Multiply 2 and 3")
thread

{'thread_id': '54dbeda0-8fa6-45ee-ab3b-5c02cac40c1c',
 'created_at': '2025-05-15T19:58:26.747406+00:00',
 'updated_at': '2025-05-15T19:58:26.747408+00:00',
 'metadata': {},
 'status': 'idle',
 'config': {},
 'values': None}

## Streaming

In [5]:
def format_tool_calls(tool_calls):
    if tool_calls:
        formatted_calls = []
        for call in tool_calls:
            formatted_calls.append(
                f"Tool Call ID: {call['id']}, Function: {call['name']}, Arguments: {call['args']}"
            )
        return "\n".join(formatted_calls)
    return "No tool calls"

In [6]:
async for event in client.runs.stream(
    thread["thread_id"],
    assistant_id="agent",
    input={"messages": [input_message]},
    stream_mode="messages",):
    
    # Handle metadata events
    if event.event == "metadata":
        print(f"Metadata: Run ID - {event.data['run_id']}")
        print("-" * 50)
    
    # Handle partial message events
    elif event.event == "messages/partial":
        for data_item in event.data:
            # Process user messages
            if "role" in data_item and data_item["role"] == "user":
                print(f"Human: {data_item['content']}")
            else:
                # Extract relevant data from the event
                tool_calls = data_item.get("tool_calls", [])
                invalid_tool_calls = data_item.get("invalid_tool_calls", [])
                content = data_item.get("content", "")
                response_metadata = data_item.get("response_metadata", {})

                if content:
                    print(f"AI: {content}")

                if tool_calls:
                    print("Tool Calls:")
                    print(format_tool_calls(tool_calls))

                if invalid_tool_calls:
                    print("Invalid Tool Calls:")
                    print(format_tool_calls(invalid_tool_calls))

                if response_metadata:
                    finish_reason = response_metadata.get("finish_reason", "N/A")
                    print(f"Response Metadata: Finish Reason - {finish_reason}")
                    
        print("-" * 50)

Metadata: Run ID - 1f031c6f-95b6-69b6-8677-885e383dacec
--------------------------------------------------
Tool Calls:
Tool Call ID: call_uCeRNIsqiVznnPUkxja24ybh, Function: multiply, Arguments: {}
--------------------------------------------------
Tool Calls:
Tool Call ID: call_uCeRNIsqiVznnPUkxja24ybh, Function: multiply, Arguments: {}
--------------------------------------------------
Tool Calls:
Tool Call ID: call_uCeRNIsqiVznnPUkxja24ybh, Function: multiply, Arguments: {}
--------------------------------------------------
Tool Calls:
Tool Call ID: call_uCeRNIsqiVznnPUkxja24ybh, Function: multiply, Arguments: {}
--------------------------------------------------
Tool Calls:
Tool Call ID: call_uCeRNIsqiVznnPUkxja24ybh, Function: multiply, Arguments: {'a': 2}
--------------------------------------------------
Tool Calls:
Tool Call ID: call_uCeRNIsqiVznnPUkxja24ybh, Function: multiply, Arguments: {'a': 2}
--------------------------------------------------
Tool Calls:
Tool Call ID: cal